In [7]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey,DateTime,Boolean,Date,Time,Float,DATE,DATETIME,TIME
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

import statsapi as mlb
from datetime import datetime
import time
import pandas as pd
import numpy as np
import plotly

In [8]:
class MyDatabase:
    # http://docs.sqlalchemy.org/en/latest/core/engines.html
    DB_ENGINE = {
       'sqlite': 'sqlite:////Users/schlinkertc/code/MLB/mlb_sqlite/mlb.db'
    }

    # Main DB Connection Ref Obj
    db_engine = None
    def __init__(self, dbtype, username='', password='', dbname=''):
        dbtype = dbtype.lower()
        if dbtype in self.DB_ENGINE.keys():
            engine_url = self.DB_ENGINE[dbtype].format(DB=dbname)
            self.db_engine = create_engine(engine_url)
            print(self.db_engine)
        else:
            print("DBType is not found in DB_ENGINE")

In [9]:
db=MyDatabase('sqlite')

Engine(sqlite:////Users/schlinkertc/code/MLB/mlb_sqlite/mlb.db)


In [10]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()
Base = declarative_base()

In [11]:
session.execute("""select count(*) from games""").fetchall()

[(141,)]

In [22]:
query=(
        """
        select 
            pitch.type_description,
            pitch.play_id,
            play.pitcher_id,
            pitch.startSpeed,
            pitch.endSpeed,
            pitch.zone,
            pitch.spinRate,
            pitch.breakLength
        from 
            pitches pitch 
            inner join plays play 
            on 
            pitch.play_id=play.id
        """
)
default_dict = {99.9:np.nan,0.0:np.nan}
pitch_df=pd.read_sql_query(query,db.db_engine)
pitch_df.replace(default_dict,inplace=True)
pitch_df.dropna(inplace=True)

In [23]:
pitch_df.describe()

,pitcher_id,startSpeed,endSpeed,zone,spinRate,breakLength
count,34710.000000,34710.000000,34710.000000,34710.000000,34710.000000,34710.000000
mean,568542.786805,88.794002,81.830141,9.917545,2265.714664,6.559585
std,66415.059071,5.894421,5.281556,4.058782,326.371900,2.840476
min,407845.000000,51.200000,46.500000,1.000000,453.000000,1.200000
25%,518876.000000,84.700000,78.300000,7.000000,2111.000000,4.800000
50%,592644.000000,89.900000,82.800000,11.000000,2288.000000,6.000000
75%,621056.000000,93.400000,85.900000,13.000000,2460.000000,8.400000
max,677976.000000,104.300000,95.600000,14.000000,3650.000000,21.600000
